# Solar Flux for the 21-cm Line
Written by Austin Lake

Saddleback College, Mission Viejo

## Introduction
This Jupyter Notebook calculates the expected solar flux at the 21-cm line, as observered from Saddleback College's 2-meter radio telescope

## Method
First, the radiance of the Sun at an estimated temperature of 5778 K, is dervied used Planck's Law of blackbody radiation. This derivation outputs units of Watts per meter-squared per sterradian [W/m^2/sr]. Because the Sun lies within the FOV of the telescope, the solid angle [sr] is calcuated from a point behind the radio telescope that propagates outward toward the Sun. The center axis of propagation passes through the both the center of the dish and the center of the Sun. Additionally, the outside edges of this solid angle form a tangent line on the edge of the dish and the meridian of the Sun that is perpendicular to the axis of propagation. The radiance from the blackbody model is then multiplied by the solid angle and the telescope's aperature area [m^2] to get the flux [W]. This is then converted to fit a logrithimic scale, resulting in units of decibel milliwatt [dBm].

In [2]:
# dependencies
from astropy import units as u
from astropy.modeling.physical_models import BlackBody
from astropy.constants import au, R_sun
from astropy.visualization import quantity_support
import numpy as np
import matplotlib.pyplot as plt

## Solid Angle Relationship
![https://doi.org/10.1117/3.2315931.ch78](images/solid_angle.jpg)

### Derving the Solid Angle of Our Dish and the Sun
$\Omega=\frac{A}{x^2}, 0\leq\Omega\leq 4\pi$

$\Omega=\frac{A_{sun}}{x^2}, A_{sun}=\mathrm{source\space area}, x=\mathrm{distance\space between\space sun\space and\space dish}$

$\Omega=\frac{A_{sun}}{X^2}=\frac{2\pi (r_{sun})^2}{x^2}=\mathrm{\frac{2\pi (695700000.0\space m)^2}{(149597870700.0\space m)^2}=0.00013588547880587752\space sr}$

In [3]:
# calculate solid angle
dishDiameter = 4.5*u.m # diameter of telescope (m)
dishArea = (np.pi*(dishDiameter/2)**2) # area of the telescope's dish (m^2)
solarArea = 2*np.pi*R_sun**2 # surface area of solar hemisphere facing dish
solidAngle = (solarArea/(R_sun**2))*u.sr # solid angle (m)

## Black Body Curve
![https://webbtelescope.org/contents/media/images/01F8GF8WYBCQVKTGPX3MA58182](images/blackbody_curves.jpg)

### Planck's Law of Blackbody Radiation

$\mathrm{specific intensity:\space}B(v,t)=\frac{2hv^3}{c^2}\cdot\frac{1}{\frac{hv}{e^{k_BT}}-1}$

$\mathrm{flux\space density = specific intensity\cdot solid\space angle}$

$\mathrm{1\space W/m^2/Hz=10^26\space Jy=10^24\space sfu}$

$\mathrm{flux = flux\space density\cdot frequency}$

$\mathrm{power = flux\cdot apeture}$

$\mathrm{1\space W = 0\space dBm\space (logarithmic\space scale)}$

In [4]:
H1 = 1420405751.768*u.Hz # Hz
temperature = 5778*u.K # temperature(K)
scale = 1.0*u.W/u.m**2/u.sr/u.Hz # blackbody scale factor (W/m^2/sr/Hz)
bb = BlackBody(temperature=temperature,scale=scale) # black body model
H1specificIntensity = bb(H1) # specific intensity (W/m^2/sr/Hz)
H1fluxDensity = H1specificIntensity*solidAngle # flux density (W/m^2/Hz)
H1jansky = H1fluxDensity.to(u.Jy) # flux density (Jy)
H1sfu = H1jansky.value*10e4 # spectral flux density (sfu)
H1flux = H1fluxDensity*H1 # flux (W/m^2)
H1power = H1flux*dishArea # power (W)
H1powerDecibel = u.Decibel(H1power, u.DecibelUnit(u.mW)) # power (dBm)

In [5]:
# display
print(f'Specific Intensity (W/m^2/sr/Hz):\t{H1specificIntensity}')
print(f'Flux Density (W/m^2/Hz):\t\t\t{H1fluxDensity}')
print(f'Flux Density (Jy):\t\t\t\t\t{H1jansky}')
print(f'Flux Density (sfu):\t\t\t\t\t{H1sfu} sfu')
print(f'Flux (W/m^2):\t\t\t\t\t\t{H1flux}')
print(f'Power (W):\t\t\t\t\t\t\t{H1power}')
print(f'Power (dBm):\t\t\t\t\t\t{H1powerDecibel}')


Specific Intensity (W/m^2/sr/Hz):	3.5815555594549246e-18 W / (Hz m2 sr)
Flux Density (W/m^2/Hz):			2.2503577268014545e-17 W / (Hz m2)
Flux Density (Jy):					2250357726.801454 Jy
Flux Density (sfu):					225035772680145.4 sfu
Flux (W/m^2):						3.196421058684347e-08 W / m2
Power (W):							5.083688038594618e-07 W
Power (dBm):						-32.93821107857174 dB(mW)
